In [ ]:
import sys
import os
import asyncio
import time
import numpy as np
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

sys.path.append(os.path.dirname(os.getcwd()))

from core.production_renaissance_system import RenaissanceProductionSystem
from config.unified_config import global_config
from data.production_database_manager import db_manager
from models.model_inference import model_inference
from scanners.real_production_scanner import production_scanner
from executors.executor_v3 import real_executor

renaissance_system = RenaissanceProductionSystem()

async def initialize_system():
    await db_manager.initialize()
    await model_inference.initialize()
    await production_scanner.initialize()
    await real_executor.initialize()
    success = await renaissance_system.initialize_system()
    return success

async def run_trading_session(duration_hours):
    await renaissance_system.start_production_trading(duration_hours)
    
print("Renaissance Trading System Initialized")

In [ ]:
config_widget = widgets.VBox([
    widgets.HTML("<h2>🚀 Renaissance Trading Control Panel</h2>"),
    widgets.FloatSlider(value=24.0, min=0.1, max=168.0, description='Duration (hours):'),
    widgets.Dropdown(options=['ethereum', 'arbitrum', 'polygon'], value='arbitrum', description='Primary Chain:'),
    widgets.FloatText(value=10.0, description='Starting Capital:'),
    widgets.FloatText(value=1.0, description='Max Position Size:'),
    widgets.Button(description='🎯 START TRADING', button_style='success'),
    widgets.Button(description='🛑 STOP TRADING', button_style='danger'),
    widgets.Output()
])

display(config_widget)

In [ ]:
await initialize_system()
await run_trading_session(24.0)

In [ ]:
performance_stats = await db_manager.get_performance_summary()
system_stats = renaissance_system.get_system_statistics()

display(HTML(f"""
<div style='background: linear-gradient(45deg, #1e3c72, #2a5298); padding: 20px; border-radius: 15px; color: white;'>
    <h2>📊 Renaissance Performance Dashboard</h2>
    <div style='display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin-top: 15px;'>
        <div><strong>Total Trades:</strong> {performance_stats.get('total_trades', 0)}</div>
        <div><strong>Win Rate:</strong> {performance_stats.get('win_rate', 0):.2%}</div>
        <div><strong>Total P&L:</strong> ${performance_stats.get('total_pnl', 0):.4f}</div>
        <div><strong>Tokens Scanned:</strong> {system_stats.get('tokens_scanned', 0):,}</div>
        <div><strong>Signals Generated:</strong> {system_stats.get('signals_generated', 0):,}</div>
        <div><strong>Portfolio Value:</strong> ${system_stats.get('portfolio_value', 0):.4f}</div>
    </div>
</div>
"""))